# 环境代码

In [1]:
import openai
import json

base_url = "http://172.16.70.15/ollama"
# model = "phi4"
model = "qwen2.5:14b"
temperature=0
max_tokens=8000

client = openai.Client(
    base_url=f"{base_url}/v1",
    api_key="ollama",  # 随便设置，本地不需要实际的key
)



## 定义文本

In [2]:
file_path = './docs/高金星等三人贩卖毒品罪、非法持有毒品罪一审刑事判决书.md'

content = open(file_path, 'r', encoding = 'utf - 8').read()

## 拆分函数

### 窗口滑动拆分

In [3]:
import re

def split_text_into_sentences(text: str) -> list:
    """
    将文本分割成句子
    
    Args:
        text (str): 输入的中文文本
        
    Returns:
        list: 句子列表
    """
    # 常见的中文句子结束符
    sentence_ends = ['。', '！', '？', '\n']
    sentences = []
    start = 0
    
    for i, char in enumerate(text):
        if char in sentence_ends:
            sentence = text[start:i + 1]
            if sentence:  # 忽略空句子
                sentences.append(sentence)
            start = i + 1
            
    # 处理最后一个句子
    if start < len(text):
        last_sentence = text[start:].strip()
        if last_sentence:
            sentences.append(last_sentence)
            
    return sentences

def split_sentences_with_sliding_window(sentences: list, window_size: int = 12, step_size: int = 10) -> list:
    """
    使用滑动窗口方式将句子列表组合成文本块
    
    Args:
        sentences (list): 句子列表
        window_size (int): 窗口大小（每个chunk包含的句子数）
        step_size (int): 滑动步长（每次移动的句子数）
        
    Returns:
        list: 包含所有文本块的列表
    """
    if not sentences:
        return []
        
    chunks = []
    start = 0
    
    while start < len(sentences):
        # 获取当前窗口的结束位置
        end = min(start + window_size, len(sentences))
        
        # 将当前窗口内的句子组合成块
        chunk = ''.join(sentences[start:end])
        chunks.append(chunk)
        
        # 如果已经处理到最后，退出循环
        if end == len(sentences):
            break
            
        # 移动窗口
        start += step_size
        
    return chunks

### 字符数拆分

In [43]:
def split_chinese_text(text: str, max_chunk_size: int = 500) -> list[str]:
    """
    将中文文本分割成固定大小的chunks，确保每个chunk包含完整的句子。
    
    Args:
        text: 输入的中文文本
        max_chunk_size: 每个chunk的最大字符数，默认500
        
    Returns:
        包含所有chunks的列表
    """
    # 定义中文标点符号
    punctuations = ['。', '！', '？', '\n']
    
    # 存储所有chunks
    chunks = []
    # 当前chunk的文本
    current_chunk = ''
    # 上一个标点符号后的文本
    last_segment = ''
    
    for char in text:
        last_segment += char
        current_chunk += char
        
        # 如果遇到标点符号，检查是否需要创建新的chunk
        if char in punctuations:
            # 如果当前chunk长度超过限制
            if len(current_chunk) >= max_chunk_size:
                # 将当前chunk添加到结果中
                chunks.append(current_chunk)
                # 重置当前chunk，保留最后一个分段
                current_chunk = last_segment
            # 重置last_segment
            last_segment = ''
            
    # 处理剩余的文本
    if current_chunk:
        chunks.append(current_chunk)
        
    return chunks

# 属性提取

In [27]:
messages=[
    {
        "role": "system", 
        "content": """请提取文书属性：
- court：审理法院
- case_type：案件类型（如刑事、民事、行政等）
- time：裁判时间
- cause：案由（民事案由或刑事罪名等）
- defendants：被告（列表）
- plaintiffs：原告（列表）

使用中文回复。

严格按照JSON格式输出。
"""
    },
    {"role": "user", "content": content[:1000]},
]

print("\n=== 测试普通对话 ===")

response = client.chat.completions.create(
    model="qwen2.5:14b",
    messages=messages,
    temperature=0,
    response_format={
        'type': 'json_object'
    },
    max_tokens=max_tokens
)

print(f"回复：{response.choices[0].message.content}")


=== 测试普通对话 ===
回复：{
    "court": "吉林省长春市中级人民法院",
    "case_type": "刑事",
    "time": "2017-12-27",
    "cause": "走私、贩卖、运输、制造毒品, 非法持有毒品罪",
    "defendants": ["高金星", "周永辉", "李成鹏"]
}


# 实体提取

## 分割文本

In [4]:
sentences = split_text_into_sentences(content)
print(f"总共分割出 {len(sentences)} 个句子")

# 使用滑动窗口组合句子
content_list = split_sentences_with_sliding_window(sentences, window_size=15, step_size=12)

content_list

总共分割出 180 个句子


['高金星等三人贩卖毒品罪、非法持有毒品罪一审刑事判决书\n案\u2003\u2003由\t走私、贩卖、运输、制造毒品\n点击了解更多\n案\u2003\u2003号\t（2017）吉01刑初91号\t\t\n  \n发布日期\t2017-12-27\t浏览次数\t90\n\n吉林省长春市中级人民法院\n刑 事 判 决 书\n（2017）吉01刑初91号\n公诉机关吉林省长春市人民检察院。\n被告人高金星，出生于吉林省长春市，初中文化，个体经营者，户籍地长春市宽城区，住湖南省株洲市。曾因吸食毒品，于2010年8月30日被行政拘留十五日。曾因犯贩卖毒品罪，于2011年7月12日被判处有期徒刑十一个月，于2011年8月2日刑满释放。',
 '被告人高金星，出生于吉林省长春市，初中文化，个体经营者，户籍地长春市宽城区，住湖南省株洲市。曾因吸食毒品，于2010年8月30日被行政拘留十五日。曾因犯贩卖毒品罪，于2011年7月12日被判处有期徒刑十一个月，于2011年8月2日刑满释放。因涉嫌犯贩卖毒品罪，于2016年9月14日在株洲市被抓获后押解回长春市，同年9月17日被刑事拘留，同年9月29日被逮捕。现羁押于长春市第二看守所。\n辩护人裴培，吉林良智律师事务所律师。\n被告人周永辉，出生于吉林省长春市，初中文化，户籍地长春市绿园区，捕前暂住湖南省株洲市石峰区红旗北路168号印象华都小区8栋2501室。因涉嫌犯非法持有毒品罪，于2016年9月14日在株洲市被抓获后押解回长春市，同年9月17日被刑事拘留，同年9月29日被逮捕。现羁押于长春市第二看守所。\n辩护人张雪明、许迟，吉林从为律师事务所律师。\n被告人李成鹏，出生于吉林省长春市，中专文化，个体经营者，户籍地长春市绿园区，捕前暂住湖南省株洲市石峰区红旗北路168号印象华都小区8栋2501室。',
 '辩护人张雪明、许迟，吉林从为律师事务所律师。\n被告人李成鹏，出生于吉林省长春市，中专文化，个体经营者，户籍地长春市绿园区，捕前暂住湖南省株洲市石峰区红旗北路168号印象华都小区8栋2501室。因涉嫌犯非法持有毒品罪，于2016年9月14日在株洲市被抓获后押解回长春市，同年9月17日被刑事拘留，同年9月29日被逮捕。现羁押于长春市第二看守所。\n辩护人申显富，吉林惠胜律师事务所律师。\n吉林省长春市人民检察院以长检刑检刑诉[20

## 提取人名

In [10]:
sys_content = f"""请提取该文书中提到的所有人物姓名，要完整人名列表，不要遗漏。
严格返回json格式，将列表放置在"persons"字段中。
"""

person_set = set()

for content in content_list:

    messages=[
        {"role": "system", "content": sys_content},
        {"role": "user", "content": content},
    ]

    print("\n=== 测试普通对话 ===")

    response = client.chat.completions.create(
        model="qwen2.5:14b",
        messages=messages,
        temperature=0,
        response_format={
            'type': 'json_object'
        },
        max_tokens=max_tokens
    )

    persons = json.loads(response.choices[0].message.content)['persons']

    print(f"回复：{persons}")

    person_set = person_set | set(persons)

person_set


=== 测试普通对话 ===
回复：['邹某非', '张某甲', '王世平']

=== 测试普通对话 ===
回复：['王辉', '王某良', '郑全胜', '宋某强']

=== 测试普通对话 ===
回复：['宋某强', '周某民']

=== 测试普通对话 ===
回复：['邹某非', '张某甲', '王某良', '宋某强', '周某民', '孙某阳']

=== 测试普通对话 ===
回复：['邹某非', '张某甲', '王少玮', '李百惠', '王世平', '王辉', '郑全胜', '王某良', '宋某强', '周某民', '孙某阳', '姜某林', '张某麟', '李某业', '马某雷', '沈某成', '卢某平']

=== 测试普通对话 ===
回复：['邹某非', '张某甲', '卢某平', '王某良', '姜某林']

=== 测试普通对话 ===
回复：['邹某非', '张某甲', '卢某平', '王某良', '孙某阳', '姜某林', '邹某非', '宋某强', '周某民']

=== 测试普通对话 ===
回复：['周某民', '李某业', '张某麟', '孙某阳', '王某良', '宋某强', '马某雷']

=== 测试普通对话 ===
回复：['宋某强', '王某良', '马某雷', '周某民', '沈某成', '姜某林', '张某麟', '邹某非', '张某甲', '孙某阳']

=== 测试普通对话 ===
回复：['邹某非', '张某甲', '王某良', '宋某强', '周某民', '孙某阳']

=== 测试普通对话 ===
回复：['孙某阳', '邹某非', '姜某林', '王某良', '卢某平', '张某甲']

=== 测试普通对话 ===
回复：['张某甲', '卢某平', '王某良', '张某麟', '李某业', '宋某强', '周某民', '马某雷', '沈某成', '孙某阳', '邹某非']

=== 测试普通对话 ===
回复：['邹某非', '张某甲', '卢某平', '姜某林', '王某良']

=== 测试普通对话 ===
回复：['邹某非', '张某甲', '卢某平', '王某良', '刘某']

=== 测试普通对话 ===
回复：['邹某非', '刘某', '张某甲', '卢某平', '陈某梅

{'2016年至2018年孙某阳',
 '×××',
 '三宝子',
 '于某发',
 '刘某',
 '卢',
 '卢某平',
 '吉杨恐',
 '吉柱',
 '吉鸣正',
 '周某喜',
 '周某民',
 '四哥',
 '姜某林',
 '孙某阳',
 '孙梦某',
 '孙继炎',
 '宋某强',
 '小成子',
 '小财主',
 '小麒',
 '小龙',
 '张某乙',
 '张某甲',
 '张某麟',
 '曲吉忠',
 '朱某龙',
 '李志华',
 '李某',
 '李某业',
 '李某臣',
 '李百惠',
 '杨某东',
 '梅什么双',
 '梅某双',
 '沈某成',
 '王世平',
 '王少玮',
 '王某良',
 '王某良女朋友',
 '王辉',
 '王金某',
 '盛某',
 '老弟',
 '赵某华',
 '邵泓语',
 '邹某非',
 '郑全胜',
 '郡来亚',
 '钱多多',
 '陆某',
 '陈某梅',
 '马某雷'}

## 实体关系

In [5]:
sys_content = f"""请提取该文书中提到的所有人名及关系。
严格返回json格式，将人名列表放置在"persons"字段中，将人物关系放置在"relations"中：
- persons（抽取到的人物列表）：
    - name（人物姓名）
    - role（人物在文书中的角色）
- relations（实体关系列表）：
    - subject（主体人名，必填字段，采用"persons"中内容）
    - object（客体人名，必填字段，采用"persons"中内容）
    - relation（实体关系，必填字段）
    - event（事件经过，包含实体行为、时间、地点等事件信息）
"""

person_set = set()
content_len = len(content_list)
i = 0

for content in content_list:

    i = i+1

    messages=[
        {"role": "system", "content": sys_content},
        {"role": "user", "content": content},
    ]

    print("\n=== 测试普通对话 ===")

    response = client.chat.completions.create(
        model="qwen2.5:14b",
        messages=messages,
        temperature=0.3,
        response_format={
            'type': 'json_object'
        },
        max_tokens=max_tokens
    )

    result = json.loads(response.choices[0].message.content)

    print(f"回复{i}/{content_len}：{result}")


=== 测试普通对话 ===
回复1/15：{'persons': [{'name': '高金星', 'role': '被告人'}, {'name': '吉林省长春市人民检察院', 'role': '公诉机关'}], 'relations': [{'subject': '高金星', 'object': '吉林省长春市人民检察院', 'relation': '被起诉', 'event': '因贩卖毒品罪、非法持有毒品罪被起诉'}, {'subject': '高金星', 'object': None, 'relation': '吸毒史', 'event': '曾于2010年8月30日因吸食毒品被行政拘留十五日'}, {'subject': '高金星', 'object': None, 'relation': '犯罪记录', 'event': '曾因犯贩卖毒品罪，于2011年7月12日被判处有期徒刑十一个月，并于2011年8月2日刑满释放'}]}

=== 测试普通对话 ===


KeyboardInterrupt: 

## 发生事件

In [39]:
sys_content = f"""请提取该文书中提到事件。
严格返回json格式，将事件列表放置在"events"字段中，每个事件中包括：
- person：字符串，事件主体姓名
- action：字符串，具体事件内容，包含行为、时间、地点等信息
"""

person_set = set()
content_len = len(content_list)
i = 0

for content in content_list:

    i = i+1

    messages=[
        {"role": "system", "content": sys_content},
        {"role": "user", "content": content},
    ]

    print("\n=== 测试普通对话 ===")

    response = client.chat.completions.create(
        model="qwen2.5:14b",
        messages=messages,
        temperature=0,
        response_format={
            'type': 'json_object'
        },
        max_tokens=max_tokens
    )

    result = json.loads(response.choices[0].message.content)

    print(f"回复{i}/{content_len}：{result}")


=== 测试普通对话 ===
回复1/9：{'events': [{'subject': '满国忠', 'action': '2018年1月3日，在四川省成都市向二姐购买甲基苯丙胺，并将毒品藏匿于火锅底料包装箱内，通过德邦物流邮寄至吉林省长春市'}]}

=== 测试普通对话 ===
回复2/9：{'events': [{'subject': '满国忠', 'action': '2018年1月3日，在四川省成都市向“二姐”购买甲基苯丙胺，并将毒品藏匿于火锅底料包装箱内，通过德邦物流邮寄至吉林省长春市'}]}

=== 测试普通对话 ===
回复3/9：{'events': [{'subject': '辩护人王向新', 'action': '提出被告人满国忠购买毒品后通过物流运输，应构成非法持有毒品罪；被告人认罪态度好'}, {'subject': '2018年1月3日', 'action': '被告人满国忠将甲基苯丙胺藏匿于火锅底料包装箱内，通过德邦物流从四川省成都市邮寄至吉林省长春市'}, {'subject': '2018年1月8日', 'action': '满国忠在长春经济技术开发区世纪大街与浦东路交汇处德邦物流收取藏毒邮包时被公安机关抓获；公安人员在满国忠收到的火锅底料包装箱中查获四包甲基苯丙胺共953.05克；在满国忠携带的钱包内查获甲基苯丙胺片剂1.71克'}, {'subject': '2017年10月', 'action': '公安机关发现满某近期在长春市二道区有贩毒行为；经过侦查，确定满国忠有重大嫌疑，并掌握满国忠在成都市购买冰毒并将毒品混在火锅底料中封装，通过德邦物流发往长春，满国忠再乘坐飞机回到长春接收快递，然后将冰毒贩卖给下线人员'}, {'subject': '2018年1月8日', 'action': '邮包到达长春，满国忠到德邦物流收取邮包后准备上车时被公安人员抓捕；当场在满国忠的邮包中查获冰毒900余克，在满国忠钱包内查获麻古1.71克'}]}

=== 测试普通对话 ===
回复4/9：{'events': [{'subject': '满国忠', 'action': '2018年1月8日，邮包到达长春，满国忠到德邦物流收取邮包后准备上车'}, {'subject': '公安人员', 'action': '将满国忠抓捕，在满国忠的邮

# 统计

## 毒资

In [14]:
sys_content = f"""请在该裁判文书中，提取出涉及涉案毒资金额的句子。
请用json返回，将句子列表存入"sentences"字段。
"""

person_set = set()
content_len = len(content_list)
i = 0

for content in content_list:

    i = i+1

    messages=[
        {"role": "system", "content": sys_content},
        {"role": "user", "content": content},
    ]

    print("\n=== 测试普通对话 ===")

    response = client.chat.completions.create(
        model="qwen2.5:14b",
        messages=messages,
        temperature=0,
        response_format={
            'type': 'json_object'
        },
        max_tokens=max_tokens
    )

    result = json.loads(response.choices[0].message.content)

    print(f"回复{i}/{content_len}：{result}")


=== 测试普通对话 ===
回复1/15：{'sentences': ['']}

=== 测试普通对话 ===
回复2/15：{'sentences': []}

=== 测试普通对话 ===
回复3/15：{'sentences': ['2016年8月中旬，被告人周永辉与李成鹏预谋后在长春市使用周永辉名下的银行卡分多次将共计39万余元人民币汇至湖南省株洲市贩毒人员高金星指定的银行帐户用于购买毒品。']}

=== 测试普通对话 ===
回复4/15：{'sentences': ['2016年8月中旬，被告人周永辉与李成鹏预谋后在长春市使用周永辉名下的银行卡分多次将共计39万余元人民币汇至湖南省株洲市贩毒人员高金星指定的银行帐户用于购买毒品。']}

=== 测试普通对话 ===
回复5/15：{'sentences': ['被告人周永辉的辩解：我汇39万元给高金星是投资工程，并非购买毒品，案发前高金星只是将一袋东西放在印象华都小区租住处的沙发上，我不知道是毒品。']}

=== 测试普通对话 ===
回复6/15：{'sentences': ['周永辉从李某1处取得10万元后，又筹集资金共39万元去湖南省株洲市购买毒品，株洲人高金星和长春人李成鹏涉案。']}

=== 测试普通对话 ===
回复7/15：{'sentences': ['户名为周永辉，卡号为×××的中国工商银行卡自2016年8月19日至20日，分5次分别向户名为黄晓婷（即被告人高金星的妻子）的卡号分别为×××、×××、×××的三张银行卡内，转入11万、5万、5万、2.8万、14.95万，共计转款38.75万元。']}

=== 测试普通对话 ===
回复8/15：{'sentences': ['2．证人李某1证言：2016年8月，我朋友周永辉向我借了十万元钱，说搞工程。']}

=== 测试普通对话 ===
回复9/15：{'sentences': ['周永辉在长春市用自己名下的工商银行卡向我提供的我妻子黄晓婷的建行卡和工商行卡汇款五笔，近39万元，用于购买毒品。']}

=== 测试普通对话 ===
回复10/15：{'sentences': ['我准备了39万元毒资。钱是我朋友赵丛久用银行转账的方式借给我29万,李某1借给我10万元,']}

=== 测试普通对话 ===
回复11/15：

## 破产